In [ ]:
import warnings
from pathlib import Path

import catboost
import lightgbm as lgb
import mapie
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import requests
import xgboost
from IPython.display import clear_output, display
from pandas.api.types import is_numeric_dtype
from pymongo import MongoClient
from pymongoarrow.api import find_pandas_all
from sklearn import (compose, dummy, ensemble, impute, linear_model, metrics,
                     model_selection, neighbors, pipeline, preprocessing, svm,
                     tree)
from sklearn.base import BaseEstimator, TransformerMixin
from tqdm import tqdm

import creds

# Get data from MongoDB

In [ ]:
cluster = MongoClient(creds.Creds.URI)

query = {"building_condition": "Good"}
df = find_pandas_all(cluster.dev.BE_houses, None)
df

# Show choropleth data

In [ ]:
BE_provinces = requests.get(
    "https://raw.githubusercontent.com/mathiasleroy/Belgium-Geographic-Data/master/dist/polygons/be-provinces-unk-WGS84.geo.json"
).json()

aggregate = (
    df.assign(list_price=lambda df: pd.to_numeric(df.price))
    .groupby("province")
    .agg(
        list_price_count=("price", "count"),
        list_price_mean=("price", "median"),
    )
    .reset_index()
)

fig = px.choropleth(
    aggregate,
    geojson=BE_provinces,
    locations="province",
    color="list_price_mean",
    featureidkey="properties.name",
    projection="mercator",
    color_continuous_scale="Magenta",
    labels={
        "list_price_mean": "Median Price",
        "list_price_count": "Number of Observations",
    },
    hover_data={"list_price_mean": ":.3s", "province": True, "list_price_count": True},
)

fig.update_geos(
    showcountries=True, showcoastlines=True, showland=True, fitbounds="locations"
)

# Add title and labels
fig.update_layout(
    title_text="Median House Prices by Province",
    autosize=False,
    width=800,
    height=600,
    geo=dict(showframe=False, showcoastlines=False, projection_type="mercator"),
)


fig.show()

In [ ]:
def show_map(data, feature):
    BE_provinces = requests.get(
        "https://raw.githubusercontent.com/mathiasleroy/Belgium-Geographic-Data/master/dist/polygons/be-provinces-unk-WGS84.geo.json"
    ).json()
    if is_numeric_dtype(data[feature]):
        aggregate = data.groupby("province")[feature].median().reset_index()
        fig = px.choropleth(
            aggregate,
            geojson=BE_provinces,
            locations="province",
            color=feature,
            featureidkey="properties.name",
            projection="mercator",
            color_continuous_scale="Magenta",
        )

        fig.update_geos(
            showcountries=True,
            showcoastlines=True,
            showland=True,
            fitbounds="locations",
        )

        fig.update_layout(
            title_text=f"Median {feature.replace('_', ' ').title()} by Province",
            coloraxis_colorbar_title_text=f"{feature.replace('_', ' ').title()}",
            autosize=False,
            width=800,
            height=600,
            geo=dict(showframe=False, showcoastlines=False, projection_type="mercator"),
        )
        return fig


show_map(df, "construction_year")

In [ ]:
def hot_index(data):
    BE_provinces = requests.get(
        "https://raw.githubusercontent.com/mathiasleroy/Belgium-Geographic-Data/master/dist/polygons/be-provinces-unk-WGS84.geo.json"
    ).json()
    number_of_ads = (
        data.groupby("province")
        .size()
        .reset_index()
        .rename(columns={0: "number_of_ads"})
    )
    fig = px.choropleth(
        number_of_ads,
        geojson=BE_provinces,
        locations="province",
        color="number_of_ads",
        featureidkey="properties.name",
        projection="mercator",
        color_continuous_scale="RdBu_r",
    )

    fig.update_geos(
        showcountries=True, showcoastlines=True, showland=True, fitbounds="locations"
    )

    fig.update_layout(
        title_text=f"Number of Ads by Province",
        coloraxis_colorbar_title_text="Number of Ads",
        autosize=False,
        width=800,
        height=600,
        geo=dict(showframe=False, showcoastlines=False, projection_type="mercator"),
    )
    return fig


hot_index(df)

# Data processing

* Remove properties below 100k
* Split X and y
* Convert y to log10

In [ ]:
df_no_null = (
    df.drop(columns=["_id", "ad_url", "day_of_retrieval"])
    .dropna(subset="price")
    .query("price >= 100000")
)

y = np.log10(df_no_null["price"])
X = df_no_null.drop(columns="price")

## Split train and test

In [ ]:
X_temp, X_test, y_temp, y_test = model_selection.train_test_split(
    X, y, test_size=0.2, random_state=45
)

X_train, X_val, y_train, y_val = model_selection.train_test_split(
    X_temp, y_temp, test_size=0.2, random_state=45
)

print(X_train.shape, X_val.shape, X_test.shape)

## Remove outliers

In [ ]:
def identify_outliers(df: pd.DataFrame) -> pd.Series:
    """
    Identify outliers in a DataFrame.

    This function uses a Local Outlier Factor (LOF) algorithm to identify outliers in a given
    DataFrame. It operates on both numerical and categorical features, and it returns a binary
    Series where `True` represents an outlier and `False` represents a non-outlier.

    Parameters:
    - df (pd.DataFrame): The input DataFrame containing features for outlier identification.

    Returns:
    - pd.Series: A Boolean Series indicating outliers (True) and non-outliers (False).

    Example:
    ```python
    # Load your DataFrame with features (df)
    df = load_data()

    # Identify outliers using the function
    outlier_mask = identify_outliers(df)

    # Use the outlier mask to filter your DataFrame
    filtered_df = df[~outlier_mask]  # Keep non-outliers
    ```

    Notes:
    - The function uses Local Outlier Factor (LOF) with default parameters for identifying outliers.
    - Numerical features are imputed using median values, and categorical features are one-hot encoded
    and imputed with median values.
    - The resulting Boolean Series is `True` for inliers and `False` for outliers.
    """

    # Extract numerical and categorical feature names
    NUMERICAL_FEATURES = df.select_dtypes("number").columns.tolist()
    CATEGORICAL_FEATURES = df.select_dtypes("object").columns.tolist()

    # Define transformers for preprocessing
    numeric_transformer = pipeline.Pipeline(
        steps=[("imputer", impute.SimpleImputer(strategy="median"))]
    )

    categorical_transformer = pipeline.Pipeline(
        steps=[
            ("encoder", preprocessing.OneHotEncoder(handle_unknown="ignore")),
            ("imputer", impute.SimpleImputer(strategy="median")),
        ]
    )

    # Create a ColumnTransformer to handle both numerical and categorical features
    preprocessor = compose.ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, NUMERICAL_FEATURES),
            ("cat", categorical_transformer, CATEGORICAL_FEATURES),
        ]
    )

    # Initialize the LOF model
    # clf = neighbors.LocalOutlierFactor()
    clf = ensemble.IsolationForest()

    # Fit LOF to preprocessed data and make predictions
    y_pred = clf.fit_predict(preprocessor.fit_transform(df))

    # Adjust LOF predictions to create a binary outlier mask
    outlier_mask = pd.Series(y_pred) == 1

    return outlier_mask

In [ ]:
outlier_mask = identify_outliers(X_train)
X_train.index = outlier_mask.index
X_train_wo_outliers = X_train[outlier_mask]

## Fit Catboost

In [ ]:
# Extract numerical and categorical feature names
NUMERICAL_FEATURES = X_train_wo_outliers.select_dtypes("number").columns.tolist()
CATEGORICAL_FEATURES = X_train_wo_outliers.select_dtypes("object").columns.tolist()

# Define transformers for preprocessing
numeric_transformer = pipeline.Pipeline(
    steps=[("imputer", impute.SimpleImputer(strategy="median"))]
)

categorical_transformer = pipeline.Pipeline(
    steps=[
        ("encoder", preprocessing.OneHotEncoder(handle_unknown="ignore")),
        ("imputer", impute.SimpleImputer(strategy="median")),
    ]
)

# Create a ColumnTransformer to handle both numerical and categorical features
preprocessor = compose.ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, NUMERICAL_FEATURES),
        ("cat", categorical_transformer, CATEGORICAL_FEATURES),
    ]
)

In [ ]:
# Fit the preprocessor on the training data
preprocessor.fit(X_train_wo_outliers)

# Transform the training and validation data
X_train_transformed = preprocessor.transform(X_train_wo_outliers)
X_val_transformed = preprocessor.transform(X_val)
X_test_transformed = preprocessor.transform(X_test)

model = catboost.CatBoostRegressor()

# Fit the model on the transformed training data
model.fit(
    X_train_transformed,
    y_train,
    eval_set=[(X_val_transformed, y_val)],
    early_stopping_rounds=100,
    verbose=False,
    use_best_model=True,
)

mapie_model = mapie.regression.MapieRegressor(model)
# Fit the MAPIE model
mapie_model.fit(X_train_transformed, y_train)

# Make predictions with prediction intervals on the transformed validation data
y_pred, y_pis = mapie_model.predict(X_test_transformed, alpha=0.1)

## Conformal prediction

In [ ]:
# Create a DataFrame with y_valid and prediction intervals
conformal_df = 10 ** pd.DataFrame(
    {
        "y_test": y_test,
        "lower": y_pis[:, 0].flatten(),
        "upper": y_pis[:, 1].flatten(),
        "y_pred": y_pred,
    }
)

# Sort the DataFrame by y_valid
df_sorted = conformal_df.sort_values(by="y_test")

# Plot data

plt.scatter(
    range(df_sorted.shape[0]),
    df_sorted["y_pred"],
    color="red",
    label="predicted",
    alpha=0.2,
)
plt.scatter(
    range(df_sorted.shape[0]),
    df_sorted["y_test"],
    color="green",
    label="ground truth",
    alpha=0.1,
)
plt.fill_between(
    range(df_sorted.shape[0]),
    df_sorted["lower"],
    df_sorted["upper"],
    alpha=0.2,
    color="gray",
    label="Prediction Intervals",
)

plt.legend()

In [ ]:
conformal_df.round(0).tail(10)

In [ ]:
metrics.root_mean_squared_error(y_test, y_pred)

In [ ]:
plt.scatter(y_test, y_pred, alpha=0.5)

In [ ]:
# catboost_train = catboost.Pool(
#     X_train,
#     y_train,
#     cat_features=X.select_dtypes(include='object').columns.tolist(),
# )

# catboost_valid = catboost.Pool(
#     X_valid,
#     y_valid,
#     cat_features=X.select_dtypes(include='object').columns.tolist(),
# )

# model = catboost.CatBoostRegressor(
#     loss_function="RMSE",
# )
model.fit(
    catboost_train,
    eval_set=[catboost_valid],
    early_stopping_rounds=100,
    verbose=False,
    use_best_model=True,
)

# mapie_model = mapie.regression.MapieRegressor(model, method="plus")

# # fit MAPIE model
# mapie_model.fit(X_train, y_train)

# # make predictions with prediction intervals
# y_pred, y_pis = mapie_model.predict(X_valid, alpha=0.1)

# Comparing different models

In [ ]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    """
    A transformer for selecting specific columns from a DataFrame.

    This class inherits from the BaseEstimator and TransformerMixin classes from sklearn.base.
    It overrides the fit and transform methods from the parent classes.

    Attributes:
        feature_names_in_ (list): The names of the features to select.
        n_features_in_ (int): The number of features to select.

    Methods:
        fit(X, y=None): Fit the transformer. Returns self.
        transform(X, y=None): Apply the transformation. Returns a DataFrame with selected features.
    """

    def __init__(self, feature_names_in_):
        """
        Constructs all the necessary attributes for the FeatureSelector object.

        Args:
            feature_names_in_ (list): The names of the features to select.
        """
        self.feature_names_in_ = feature_names_in_
        self.n_features_in_ = len(feature_names_in_)

    def fit(self, X, y=None):
        """
        Fit the transformer. This method doesn't do anything as no fitting is necessary.

        Args:
            X (DataFrame): The input data.
            y (array-like, optional): The target variable. Defaults to None.

        Returns:
            self: The instance itself.
        """
        return self

    def transform(self, X, y=None):
        """
        Apply the transformation. Selects the features from the input data.

        Args:
            X (DataFrame): The input data.
            y (array-like, optional): The target variable. Defaults to None.

        Returns:
            DataFrame: A DataFrame with only the selected features.
        """
        return X.loc[:, self.feature_names_in_].copy(deep=True)

In [ ]:
y = np.log10(df_no_null["price"])
X = df_no_null.drop(columns=["price", "_id", "ad_url", "day_of_retrieval"])

X_train, X_valid, y_train, y_valid = model_selection.train_test_split(
    X, y, test_size=0.2
)

In [ ]:
# Selecting columns by dtypes

numerical_columns = X_train.head().select_dtypes("number").columns.to_list()
categorical_columns = X_train.head().select_dtypes("object").columns.to_list()

In [ ]:
# Prepare pipelines for corresponding columns:
numerical_pipeline = pipeline.Pipeline(
    steps=[
        ("num_selector", FeatureSelector(numerical_columns)),
        ("imputer", impute.SimpleImputer(strategy="median")),
        ("std_scaler", preprocessing.MinMaxScaler()),
    ]
)

categorical_pipeline = pipeline.Pipeline(
    steps=[
        ("cat_selector", FeatureSelector(categorical_columns)),
        ("imputer", impute.SimpleImputer(strategy="most_frequent")),
        (
            "onehot",
            preprocessing.OneHotEncoder(handle_unknown="ignore", sparse_output=True),
        ),
    ]
)

# Put all the pipelines inside a FeatureUnion:
data_preprocessing_pipeline = pipeline.FeatureUnion(
    n_jobs=-1,
    transformer_list=[
        ("numerical_pipeline", numerical_pipeline),
        ("categorical_pipeline", categorical_pipeline),
    ],
)

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter(action="ignore", category=FutureWarning)

    MLA = [
        linear_model.LinearRegression(),
        linear_model.SGDRegressor(),
        linear_model.PassiveAggressiveRegressor(),
        linear_model.RANSACRegressor(),
        linear_model.Lasso(),
        svm.SVR(),
        ensemble.GradientBoostingRegressor(),
        tree.DecisionTreeRegressor(),
        ensemble.RandomForestRegressor(),
        ensemble.ExtraTreesRegressor(),
        ensemble.AdaBoostRegressor(),
        catboost.CatBoostRegressor(silent=True),
        lgb.LGBMRegressor(verbose=-1),
        xgboost.XGBRegressor(verbosity=0),
        dummy.DummyClassifier(),
    ]

    # note: this is an alternative to train_test_split
    cv_split = model_selection.ShuffleSplit(
        n_splits=5, test_size=0.3, train_size=0.6, random_state=0
    )  # run model 10x with 60/30 split intentionally leaving out 10%

    # create table to compare MLA metrics
    MLA_columns = [
        "MLA Name",
        "MLA Parameters",
        "MLA Train RMSE Mean",
        "MLA Test RMSE Mean",
        "MLA Train R2 Mean",
        "MLA Test R2 Mean",
        "MLA Time",
    ]
    MLA_compare = pd.DataFrame(columns=MLA_columns)

    # index through MLA and save performance to table
    row_index = 0
    for alg in tqdm(MLA):
        # set name and parameters
        MLA_name = alg.__class__.__name__
        MLA_compare.loc[row_index, "MLA Name"] = MLA_name
        MLA_compare.loc[row_index, "MLA Parameters"] = str(alg.get_params())

        model_pipeline = pipeline.Pipeline(
            steps=[
                ("data_preprocessing_pipeline", data_preprocessing_pipeline),
                ("model", alg),
            ]
        )

        cv_results = model_selection.cross_validate(
            model_pipeline,
            X_train,
            y_train,
            cv=cv_split,
            scoring={
                "r2": "r2",
                "neg_root_mean_squared_error": "neg_root_mean_squared_error",
            },
            return_train_score=True,
        )

        MLA_compare.loc[row_index, "MLA Time"] = cv_results["fit_time"].mean()
        MLA_compare.loc[row_index, "MLA Train RMSE Mean"] = cv_results[
            "train_neg_root_mean_squared_error"
        ].mean()
        MLA_compare.loc[row_index, "MLA Test RMSE Mean"] = cv_results[
            "test_neg_root_mean_squared_error"
        ].mean()

        MLA_compare.loc[row_index, "MLA Train R2 Mean"] = cv_results["train_r2"].mean()
        MLA_compare.loc[row_index, "MLA Test R2 Mean"] = cv_results["test_r2"].mean()

        row_index += 1

        clear_output(wait=True)
        display(MLA_compare.sort_values(by=["MLA Test RMSE Mean"], ascending=False))